## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import scipy as sc
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile
import cv2
import librosa

# Import sci-kit models
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

## Load Data

In [8]:
# Launch Jupyter Notebook from the project folder
root_path = os.getcwd()
# Data is stored in the folder named "Data"
path = os.path.join(root_path, 'Data')

In [23]:
# Create data lists
samples = []
labels = []
classes = os.listdir(path)

for file in os.listdir(path):
    class_path = os.path.join(path, file)
    for filename in os.listdir(class_path):
        print(filename)

        # Define sample rate
        fs = 44100

        fs_data, y = wavfile.read(os.path.join(class_path, filename))
        if len(y.shape) == 2:
            y = y[:,0]
        print(fs_data)
        print(y[2570])
        print(type(y))

        if fs_data == fs:
            labels.append([file])
            samples.append(y)
        else:
            print('Warning: Data in {} not sampled at 44.1 kHz!'.format(filename))
            print(f'Sampled at {fs_data}')
            # TODO: Down sample data that is not 44.1 kHz (most seem to be 48.0 kHz)

        print()
      # break

D-bifircus-D-parviceps-LS100120.WAV
44100
-1221888
<class 'numpy.ndarray'>

D-bifurcus-sc10743.wav
44100
-95
<class 'numpy.ndarray'>

Dbifurcus2.WAV
48000
100096
<class 'numpy.ndarray'>
Sampled at 48000

Drhodopeplusbifurcus2_Daniel1.WAV
48000
79037440
<class 'numpy.ndarray'>
Sampled at 48000

LS100120.WAV
44100
-1221888
<class 'numpy.ndarray'>

LS100303 D. bifurcus sc38590mono.WAV
44100
498944
<class 'numpy.ndarray'>

LS100536.WAV
44100
15
<class 'numpy.ndarray'>

LS100582.WAV


C:\Users\14w\AppData\Local\Temp\ipykernel_26676\2532551335.py:14: WavFileWarning: Chunk (non-data) not understood, skipping it.
  fs_data, y = wavfile.read(os.path.join(class_path, filename))


44100
12234240
<class 'numpy.ndarray'>

LS100584.WAV
44100
7324416
<class 'numpy.ndarray'>

LS100594.WAV
44100
7713792
<class 'numpy.ndarray'>

1017-sc28418-E-petersi-km22mono.WAV
44100
305
<class 'numpy.ndarray'>

1019-sc28436-E-petersi-km10_Daniel1.WAV
44100
-339
<class 'numpy.ndarray'>

1020-sc-28437-E-petersi-km10MONO_Daniel1.WAV
44100
46268416
<class 'numpy.ndarray'>

1021-sc28494-E-petersi.WAV
44100
279
<class 'numpy.ndarray'>

E-petersi-km-40-sc10741.WAV
44100
0
<class 'numpy.ndarray'>

E-petersi-sc28417-km22.WAV
44100
-2309
<class 'numpy.ndarray'>

E-petesi-sc28416-km22.WAV
44100
8
<class 'numpy.ndarray'>

Epetersi2_Daniel1.WAV
48000
171103488
<class 'numpy.ndarray'>
Sampled at 48000

Epetersi3_Daniel.WAV
48000
-601856
<class 'numpy.ndarray'>
Sampled at 48000

Epetersi_Daniel1.WAV
48000
667904
<class 'numpy.ndarray'>
Sampled at 48000

LS100150 E. petersi sc28466.WAV
44100
-55030784
<class 'numpy.ndarray'>

LS100158 E. petersi no capturado_Daniel1.WAV
48000
154278400
<class 'num

In [63]:
print('Loaded data from the directory {} \n'.format(path))
print('Loaded the classes: {} \n'.format(classes))
print('Loaded {} samples with {} labels \n'.format(len(samples), len(labels)))

# Check lengths of samples
for idx, sample in enumerate(samples):
  print('Sample {:2}: \t {:.1f}s \t {}'.format(idx, len(sample)/44100, labels[idx]))

Loaded data from the directory C:\Users\14w\Documents\CS522_final\Data 

Loaded the classes: ['Dendropsophus bifurcus', 'Engystomops petersi', 'Pristimantis conspicillatus'] 

Loaded 25 samples with 25 labels 

Sample  0: 	 105.8s 	 ['Dendropsophus bifurcus']
Sample  1: 	 593.1s 	 ['Dendropsophus bifurcus']
Sample  2: 	 105.8s 	 ['Dendropsophus bifurcus']
Sample  3: 	 240.3s 	 ['Dendropsophus bifurcus']
Sample  4: 	 313.6s 	 ['Dendropsophus bifurcus']
Sample  5: 	 409.2s 	 ['Dendropsophus bifurcus']
Sample  6: 	 208.7s 	 ['Dendropsophus bifurcus']
Sample  7: 	 317.8s 	 ['Dendropsophus bifurcus']
Sample  8: 	 512.6s 	 ['Engystomops petersi']
Sample  9: 	 419.5s 	 ['Engystomops petersi']
Sample 10: 	 368.0s 	 ['Engystomops petersi']
Sample 11: 	 320.0s 	 ['Engystomops petersi']
Sample 12: 	 641.0s 	 ['Engystomops petersi']
Sample 13: 	 1018.6s 	 ['Engystomops petersi']
Sample 14: 	 504.0s 	 ['Engystomops petersi']
Sample 15: 	 356.8s 	 ['Engystomops petersi']
Sample 16: 	 264.1s 	 ['Pris

## Encode labels

In [64]:
# Reshape classes list
unique_classes = [[c] for c in classes]

# Create encoder, fit, and check encoding
enc = OneHotEncoder()
enc.fit(unique_classes)
enc_labels = enc.transform(labels).toarray()
for idx, item in enumerate(enc_labels):
    print('Encoding is {}, label is {}'.format(enc_labels[idx], labels[idx]))

Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 0. 1.], label is ['Pristimantis conspicillatus']

## Get Spectrograms

In [65]:
def get_spectrograms(time_data, FFT_SIZE=1024, fs=44100):
    ## Compute Fast Fourier Transform of data
    # Create FFT lists
    pxx = []
    
    print("Calculating FFTs for data ...")
    for sample in time_data:
        # flatten() is used because some of the data is stereo rather than mono. This results in having
        # two data points for each data point rather than 1, and something like (# data points, 2) vs. (# data points, ).
        # flatten(), flattens out the second dimension, resulting in (2x # data points, )
#             if windows is True:
        _, _, pxx_idx = signal.spectrogram(sample, nperseg=FFT_SIZE, fs=fs, noverlap=FFT_SIZE/2)
        
        pxx.append(pxx_idx)
                            
    print('Calculated all FFTs')

    return pxx.copy()

In [66]:
specs = get_spectrograms(samples)

Calculating FFTs for data ...
Calculated all FFTs


In [67]:
print(len(specs))
print(len(specs[0]))
print(len(specs[0][0]))

25
513
9111


## Binning Data

In [68]:
def get_bins(spec_data, num_freq_bins=5, num_time_bins=5):
    binned_data = []
        
    # Get individual samples
    for s in spec_data:
        #Open CV's resize takes (columns,rows) as the input for desired size
        resized_pxx=cv2.resize(s[:,:],(num_time_bins,num_freq_bins))
        binned_data.append(resized_pxx.flatten())
            
    return binned_data

In [69]:
binned_data = get_bins(specs)

In [70]:
b = np.array(binned_data)
print(b.shape)

(25, 25)


## Get Freq-Windows

In [71]:
def get_freq_windows(time_data, len_window=1, overlap=0, sample_rate=10):
    len_data = time_data.shape[1]
#     print(len_data)
    
    windows = []
    
    # How far the window should shift
    if overlap == 0:
        step = len_window
        
        start = 0
        stop = step
#         print(len_window)
        for n in range(int(len_data/len_window)):
            windows.append(time_data[:, start:stop])
            
#             print(start)
#             print(stop)
#             print()
            start = start + step
            stop = stop + step
            
            if start > len_data or stop > len_data:
                break
    else:
        step = int(len_window*overlap)
        
        start = 0
        stop = len_window
        
#         print(len_window)
        for n in range(int(len_data/step)):
            windows.append(time_data[:, start:stop])
            
#             print(start)
#             print(stop)
#             print()
            start = start + step
            stop = stop + step
            
            if start > len_data or stop > len_data:
                break
        
    return windows

In [72]:
windowed_freq = []
for fft in specs:
    window_freq_data = get_windows(fft, len_window=2, overlap=.5)
    #shape = np.array(window_data).shape
    #window_data = np.array(window_data).reshape(shape[0], shape[1]*shape[2]) 
    print(np.array(window_data).shape)
    windowed_freq.append(window_data)  

NameError: name 'get_windows' is not defined

## Get Time-Windows

In [73]:
def get_time_windows(time_data, len_window_sec=1, overlap=0):
  len_data = time_data.shape[0]
  # print(len_data)

  len_window = len_window_sec*44100
    
  windows = []
  # How far the window should shift
  if overlap == 0:
      step = len_window
      
      start = 0
      stop = step
      # print(len_window)

      for n in range(int(len_data/len_window)):
          windows.append(time_data[start:stop])
          
          # print(start)
          # print(stop)
          # print()
          start = start + step
          stop = stop + step
          
          if start > len_data or stop > len_data:
              break
  else:
      step = int(len_window*overlap)
      
      start = 0
      stop = len_window
      
  #         print(len_window)
      for n in range(int(len_data/step)):
          windows.append(time_data[start:stop])
          
  #             print(start)
  #             print(stop)
  #             print()
          start = start + step
          stop = stop + step
          
          if start > len_data or stop > len_data:
              break
        
  return windows

time_windows = []
num_labels = []
for sample in samples:
  new_windows = get_time_windows(samples[0], len_window_sec=5)
  num_labels.append(len(new_windows))
  print(np.array(new_windows).shape)
  time_windows = time_windows + new_windows

(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)
(21, 220500)


In [74]:
new_labels = []
for label in labels:
  new_labels = new_labels + [label for num in num_labels]

enc_labels = enc.transform(new_labels).toarray()
print('Encoding is:')
print(enc_labels)

Encoding is:
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [75]:
train, test = cv.split(np.array(X), np.array(y.argmax(axis=1)))

NameError: name 'cv' is not defined

## Feature Extraction

In [76]:
specs_reshape = []
for i in range(len(specs)):
  shape = np.array(specs[i]).shape
  X = np.array(specs[i]).reshape(shape[0]*shape[1])
  specs_reshape.append(X)
  

In [83]:
window_numbers = 20 ## need to be determined
domain_fv = []

for i in range(len(specs_reshape)):
  #specs reshape: [frequency][time]
  print(specs_reshape[0])
  
    # peak frequency
      for 

    # zero crossing rate
    # threshold crossing rate bandwidth

    # MFCC
    mfccs = librosa.feature.mfcc(y=specs_reshape[i], sr=44100, n_mfcc=20, win_length = int(np.ceil(1024/window_numbers)), hop_length = int(np.ceil(1024/(2*window_numbers))))
    mfccs_mean = np.mean(mfccs.T, axis=0)
    #mfccs_mean=mfccs_mean.reshape(513,20)

    # Spectral Centroid
#     sc = librosa.feature.spectral_centroid(y=spectrograms[i], sr=44100, win_length = int(np.ceil(1024/5)), hop_length = int(np.ceil(1024/10)))
#     reshape_sc=[]
#     for x in sc:
#         for j in x:
#             reshape_sc.append(j)

    #all_features=list(zip(mfccs_mean,reshape_sc))
    domain_fv.append(mfccs_mean)
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 15)

## Train Model

In [79]:
# Specify which data to use, these are the only parameters that should change, the rest should remain the same.
X = time_windows
y = enc_labels
depth = 5


# # Create our imputer to replace missing values with the mean 
# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp = imp.fit(X)
    
# # Impute our data
# X_imp = imp.transform(X)

# Define model
clf = RandomForestClassifier(max_depth=depth, random_state=0)

# Define number of folds
cv = StratifiedKFold(n_splits=10, shuffle=False)

# Split data, train and test model on 10 folds
split = 1
scores = []
confuse_mat = []
for train_index, test_index in cv.split(np.array(X), np.array(y.argmax(axis=1))):
    print(f"Split {split}/10...")
    x_train, y_train = X[train_index], enc_labels[train_index]
    x_test, y_test = X[test_index], enc_labels[test_index]
    
    
    # Fit model and evaluate it
    clf.fit(x_train, y_train)
    scores.append(clf.score(x_test, y_test))
    
    # Construct confusion matrix
    y_predict = clf.predict(x_test)
    confuse_mat.append(confusion_matrix(y_test.argmax(axis=1), y_predict.argmax(axis=1)))
    
    # Iterate
    split = split + 1
    
print("Mean accuracy:" + str(np.mean(scores)))

ValueError: Found input variables with inconsistent numbers of samples: [525, 625]

In [ ]:
# TODO: should probably do kfold but doing this just for testing purposes
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(binned_data, enc_labels, test_size=.3, stratify=enc_labels)

In [ ]:
clf = RandomForestClassifier()

clf.fit(x_train, y_train)

print(clf.score(x_test, y_test))

## Save Model

In [ ]:
import pickle

pkl_filename = "pickle_model.pkl"

# Note, for now this will only store the file in volitle memory in the notebook
# To save permanently, most add file path to drive
with open(pkl_filename, 'wb') as file:
  pickle.dump(clf, file)

## Load Model

In [ ]:
with open(pkl_filename, 'rb') as file:
  pickle_model = pickle.load(file)

In [ ]:
# Test saved model
print(pickle_model.score(x_test, y_test))